## posts with all features 


In [11]:
import os
import requests
import pandas as pd
from datetime import datetime
from tqdm import tqdm

# Reddit API credentials
client_id = 'AIgVuy1vju3sPYddN7fZYA'
secret_token = '4vS8QLhVdf8J_v4xaSgnTj7k1Sl9tg'
username = 'BriefWall2565'
password = 'C@Jq,Byn@mGi-6F'

# Parameters
limit = 100  # Adjust the number of posts per request
num_iter = 10  # Adjust the number of iterations per subreddit
run_dir='run3'
os.makedirs(f'posts_v2/{run_dir}',exist_ok=True)
# Ensure output folder exists
os.makedirs("posts", exist_ok=True)

# Function to convert API responses to DataFrame
def df_from_response(res):
    try:
        response_json = res.json()
        if not isinstance(response_json, dict) or 'data' not in response_json:
            print(f"Unexpected response structure: {response_json}")
            return pd.DataFrame()  # Return empty DataFrame
        
        posts = []
        for post in response_json['data'].get('children', []):
            post_data = post['data']
            posts.append({
                'subreddit': post_data.get('subreddit'),
                'title': post_data.get('title'),
                'selftext': post_data.get('selftext'),
                'author': post_data.get('author'),
                'id': post_data.get('id'),
                'permalink': f"https://reddit.com{post_data.get('permalink')}",
                'url': post_data.get('url'),
                'created_utc': datetime.utcfromtimestamp(post_data.get('created_utc')).strftime('%Y-%m-%d %H:%M:%S'),
                'score': post_data.get('score'),
                'upvote_ratio': post_data.get('upvote_ratio'),
                'ups': post_data.get('ups'),
                'downs': post_data.get('downs'),
                'num_comments': post_data.get('num_comments'),
                'total_awards_received': post_data.get('total_awards_received'),
                'gilded': post_data.get('gilded'),
                'is_video': post_data.get('is_video'),
                'is_original_content': post_data.get('is_original_content'),
                'is_self': post_data.get('is_self'),
                'over_18': post_data.get('over_18'),
                'spoiler': post_data.get('spoiler'),
                'link_flair_text': post_data.get('link_flair_text'),
                'thumbnail': post_data.get('thumbnail'),
            })
        return pd.DataFrame(posts)
    except Exception as e:
        print(f"Error processing response: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error




# Authenticate API and get OAuth token
def authenticate():
    client_auth = requests.auth.HTTPBasicAuth(client_id, secret_token)
    auth_data = {'grant_type': 'password', 'username': username, 'password': password}
    headers = {'User-Agent': 'myBot/0.0.1'}

    auth_res = requests.post('https://www.reddit.com/api/v1/access_token',
                             auth=client_auth, data=auth_data, headers=headers)
    if auth_res.status_code == 200:
        token = f"bearer {auth_res.json().get('access_token')}"
        headers['Authorization'] = token
        return headers
    else:
        raise Exception(f"Authentication failed! Status: {auth_res.status_code}, Message: {auth_res.text}")

# Subreddit list
subreddits = [
    "python", "webdev", "MachineLearning", "datascience", "artificial",
    "programming", "devops", "physics", "math", "science", "learnprogramming",
    "learnpython", "gaming", "pcgaming", "books", "movies", "photography",
    "music", "productivity", "careerguidance", "personalfinance", "AskReddit",
    "todayilearned", "interestingasfuck",
    "ReverseEngineering",  # Hacking and retrofitting tools
    "Technology",          # Tech news and innovations
    "Hacking",             # Ethical hacking and Linux discussions
    "GetMotivated",        # Inspirational stories and quotes
    "LifeProTips",         # Tips and hacks for everyday life
    "unpopularopinion",    # Share unique and debated opinions
    "worldnews",           # Global news coverage
    "pics"                 # Viral photos and casual photography
]
tags=['hot','top','rising','controversial','search','random']  # and 'new' 
# Fetch posts
def fetch_posts(headers, subreddits, num_iter, limit):
    for tag in tqdm(tags):
        for sub in subreddits:
            print(f"Fetching posts for r/{sub} with tag {tag}")
            params = {'limit': limit}
            data = pd.DataFrame()
            for _ in tqdm(range(num_iter)):
                res = requests.get(f"https://oauth.reddit.com/r/{sub}/{tag}",
                                   headers=headers, params=params)
    
                if res.status_code != 200:
                    print(f"Request failed for r/{sub}: {res.status_code}")
                    break
    
                new_df = df_from_response(res)
                if new_df.empty:
                    print(f"No more data to fetch for r/{sub}.")
                    break
    
                data = pd.concat([data, new_df], ignore_index=True)
                last_row = new_df.iloc[-1]
                # print(f"Last row: {last_row}")  # Debugging info
                # Use 'name' for pagination
                params['after'] = last_row.get('name')  # 'name' is the full identifier (kind_id)
    
            # Drop unnecessary columns and save to CSV
            data.drop(columns=['kind'], inplace=True, errors='ignore')
            data.to_csv(f'posts_v2/{run_dir}/{sub}_{tag}.csv', index=False)
            print(f"Saved posts for r/{sub} to posts_v2/{run_dir}/{sub}_{tag}.csv")


# Main Execution
if __name__ == "__main__":
    headers = authenticate()
    fetch_posts(headers, subreddits, num_iter, limit)


  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

Fetching posts for r/python with tag hot



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:23<00:00,  2.31s/it]


Saved posts for r/python to posts_v2/run3/python_hot.csv
Fetching posts for r/webdev with tag hot



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.98s/it]


...

Saved posts for r/python to posts_v2/run3/python_controversial.csv
Fetching posts for r/webdev with tag controversial



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.08s/it]


Saved posts for r/webdev to posts_v2/run3/webdev_controversial.csv
Fetching posts for r/MachineLearning with tag controversial



 50%|█████████████████████████████████████████▌                                         | 3/6 [20:21<20:21, 407.23s/it]


ConnectionError: HTTPSConnectionPool(host='oauth.reddit.com', port=443): Max retries exceeded with url: /r/MachineLearning/controversial?limit=100 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000022BE5EE8A90>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

## comments

In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime
from tqdm import tqdm

# Reddit API credentials
client_id = 'AIgVuy1vju3sPYddN7fZYA'
secret_token = '4vS8QLhVdf8J_v4xaSgnTj7k1Sl9tg'
username = 'BriefWall2565'
password = 'C@Jq,Byn@mGi-6F'

# Parameters
limit = 100  # Adjust the number of posts per request
num_iter = 10  # Adjust the number of iterations per subreddit
run_dir='run3'
os.makedirs(f'posts_v2/{run_dir}',exist_ok=True)
# Ensure output folder exists
os.makedirs("posts", exist_ok=True)

# Function to convert API responses to DataFrame
def df_from_response(res):
    try:
        response_json = res.json()
        if not isinstance(response_json, dict) or 'data' not in response_json:
            print(f"Unexpected response structure: {response_json}")
            return pd.DataFrame()  # Return empty DataFrame
        
        posts = []
        for post in response_json['data'].get('children', []):
            post_data = post['data']
            posts.append({
                'subreddit': post_data.get('subreddit'),
                'title': post_data.get('title'),
                'selftext': post_data.get('selftext'),
                'author': post_data.get('author'),
                'id': post_data.get('id'),
                'permalink': f"https://reddit.com{post_data.get('permalink')}",
                'url': post_data.get('url'),
                'created_utc': datetime.utcfromtimestamp(post_data.get('created_utc')).strftime('%Y-%m-%d %H:%M:%S'),
                'score': post_data.get('score'),
                'upvote_ratio': post_data.get('upvote_ratio'),
                'ups': post_data.get('ups'),
                'downs': post_data.get('downs'),
                'num_comments': post_data.get('num_comments'),
                'total_awards_received': post_data.get('total_awards_received'),
                'gilded': post_data.get('gilded'),
                'is_video': post_data.get('is_video'),
                'is_original_content': post_data.get('is_original_content'),
                'is_self': post_data.get('is_self'),
                'over_18': post_data.get('over_18'),
                'spoiler': post_data.get('spoiler'),
                'link_flair_text': post_data.get('link_flair_text'),
                'thumbnail': post_data.get('thumbnail'),
            })
        return pd.DataFrame(posts)
    except Exception as e:
        print(f"Error processing response: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error




# Authenticate API and get OAuth token
def authenticate():
    client_auth = requests.auth.HTTPBasicAuth(client_id, secret_token)
    auth_data = {'grant_type': 'password', 'username': username, 'password': password}
    headers = {'User-Agent': 'myBot/0.0.1'}

    auth_res = requests.post('https://www.reddit.com/api/v1/access_token',
                             auth=client_auth, data=auth_data, headers=headers)
    if auth_res.status_code == 200:
        token = f"bearer {auth_res.json().get('access_token')}"
        headers['Authorization'] = token
        return headers
    else:
        raise Exception(f"Authentication failed! Status: {auth_res.status_code}, Message: {auth_res.text}")

# Subreddit list
subreddits = [
    "python", "webdev", "MachineLearning", "datascience", "artificial",
    "programming", "devops", "physics", "math", "science", "learnprogramming",
    "learnpython", "gaming", "pcgaming", "books", "movies", "photography",
    "music", "productivity", "careerguidance", "personalfinance", "AskReddit",
    "todayilearned", "interestingasfuck",
    "ReverseEngineering",  # Hacking and retrofitting tools
    "Technology",          # Tech news and innovations
    "Hacking",             # Ethical hacking and Linux discussions
    "GetMotivated",        # Inspirational stories and quotes
    "LifeProTips",         # Tips and hacks for everyday life
    "unpopularopinion",    # Share unique and debated opinions
    "worldnews",           # Global news coverage
    "pics"                 # Viral photos and casual photography
]
tags=['hot','top','rising','controversial','search','random']  # and 'new' 
# Fetch posts
def fetch_comments(headers, subreddit, post_id):
    """
    Fetches comments for a given Reddit post.
    
    :param headers: Authentication headers
    :param subreddit: The subreddit name
    :param post_id: The ID of the post
    :return: DataFrame containing comments
    """
    url = f"https://oauth.reddit.com/r/{subreddit}/comments/{post_id}"
    params = {'limit': 100}  # You can adjust the limit as needed

    res = requests.get(url, headers=headers, params=params)
    if res.status_code != 200:
        print(f"Failed to fetch comments for post {post_id} in r/{subreddit}: {res.status_code}")
        return pd.DataFrame()

    try:
        response_json = res.json()
        if not isinstance(response_json, list) or len(response_json) < 2:
            print(f"Unexpected response structure: {response_json}")
            return pd.DataFrame()

        comments = []
        # Comments are in the second element of the JSON array
        for comment in response_json[1]['data']['children']:
            if 'data' in comment:
                comment_data = comment['data']
                comments.append({
                    'post_id': post_id,
                    'comment_id': comment_data.get('id'),
                    'author': comment_data.get('author'),
                    'body': comment_data.get('body'),
                    'created_utc': datetime.utcfromtimestamp(comment_data.get('created_utc')).strftime('%Y-%m-%d %H:%M:%S'),
                    'score': comment_data.get('score'),
                    'parent_id': comment_data.get('parent_id'),
                    'is_submitter': comment_data.get('is_submitter'),
                })
        return pd.DataFrame(comments)
    except Exception as e:
        print(f"Error processing comments for post {post_id}: {e}")
        return pd.DataFrame()

# Main Execution
if __name__ == "__main__":
    headers = authenticate()
    fetch_comments(headers, subreddits, num_iter, limit)
